# Cluster the Spire
Will Wright

In [35]:
import shutil
from os import listdir
import json
import glob

In [31]:
def extract_all(archives, extract_path, zip_format = "gztar"):
    '''
    input: path to zipped file archives, path to extract, and type of zipped file
    output: unzipped contents of each zipped file withine the extract path
    '''
    for filename in listdir(archives):
        shutil.unpack_archive(archives+filename, extract_path, zip_format)

In [32]:
extract_all("../data_raw/zipped/","../data_raw/unzipped/", "gztar")

In [74]:
read_files = glob.glob("../data_raw/unzipped/*/*.json", recursive = True)

In [84]:
read_files[5583]

'../data_raw/unzipped/2019-05-19/20190519195200_1553800255.run.json'

In [100]:
# this approach creates one mega json
with open("merged_file.json", "wt") as outfile:
    outfile.write('[{}]'.format(','.join([open(f, "rt").read() for f in read_files[0:9]])))

In [152]:
read_files.index("../data_raw/unzipped/2019-06-06/20190606113929_autoupload.json")

10025

In [154]:
read_files.index(matching)

ValueError: ['../data_raw/unzipped/2019-06-06/20190606113929_autoupload.json', '../data_raw/unzipped/2019-06-06/20190606113845_autoupload.json', '../data_raw/unzipped/2019-06-06/20190606113803_autoupload.json', '../data_raw/unzipped/2019-06-06/20190606113619_autoupload.json', '../data_raw/unzipped/2019-06-06/20190606113912_autoupload.json', '../data_raw/unzipped/2019-06-06/20190606113409_autoupload.json', '../data_raw/unzipped/2019-06-06/20190606120450_autoupload.json', '../data_raw/unzipped/2019-06-06/20190606113208_autoupload.json', '../data_raw/unzipped/2019-06-06/20190606113001_autoupload.json', '../data_raw/unzipped/2019-06-06/20190606113446_autoupload.json', '../data_raw/unzipped/2019-06-06/20190606120434_autoupload.json', '../data_raw/unzipped/2019-06-06/20190606113433_autoupload.json', '../data_raw/unzipped/2019-06-06/20190606112906_autoupload.json', '../data_raw/unzipped/2019-06-06/20190606123015_autoupload.json', '../data_raw/unzipped/2019-06-06/20190606113152_autoupload.json

In [147]:
matching = [s for s in read_files if "autoupload" in s]

In [148]:
matching

['../data_raw/unzipped/2019-06-06/20190606113929_autoupload.json',
 '../data_raw/unzipped/2019-06-06/20190606113845_autoupload.json',
 '../data_raw/unzipped/2019-06-06/20190606113803_autoupload.json',
 '../data_raw/unzipped/2019-06-06/20190606113619_autoupload.json',
 '../data_raw/unzipped/2019-06-06/20190606113912_autoupload.json',
 '../data_raw/unzipped/2019-06-06/20190606113409_autoupload.json',
 '../data_raw/unzipped/2019-06-06/20190606120450_autoupload.json',
 '../data_raw/unzipped/2019-06-06/20190606113208_autoupload.json',
 '../data_raw/unzipped/2019-06-06/20190606113001_autoupload.json',
 '../data_raw/unzipped/2019-06-06/20190606113446_autoupload.json',
 '../data_raw/unzipped/2019-06-06/20190606120434_autoupload.json',
 '../data_raw/unzipped/2019-06-06/20190606113433_autoupload.json',
 '../data_raw/unzipped/2019-06-06/20190606112906_autoupload.json',
 '../data_raw/unzipped/2019-06-06/20190606123015_autoupload.json',
 '../data_raw/unzipped/2019-06-06/20190606113152_autoupload.js

In [120]:
# this approach creates a list of 
output_list = []

for f in read_files:
    if not output_list[f]
    with open(f, "r", errors = 'ignore') as infile: #ignore parsing errors where there are unparsable characters
        output_list.append(json.load(infile))
    

with open("merged_file.json", "w") as outfile:
    json.dump(output_list, outfile)

../data_raw/unzipped/2019-09-14/20190914222939_1567367529.run.json
../data_raw/unzipped/2019-09-14/20190914055039_1568090686.run.json
../data_raw/unzipped/2019-09-14/20190914161409_1565665826.run.json
../data_raw/unzipped/2019-09-14/20190914142609_1567876755.run.json
../data_raw/unzipped/2019-09-14/20190914125259_1568479920.run.json
../data_raw/unzipped/2019-09-14/20190914222940_1567475651.run.json
../data_raw/unzipped/2019-09-14/20190914161421_1565485112.run.json
../data_raw/unzipped/2019-09-14/20190914223203_1567571078.run.json
../data_raw/unzipped/2019-09-14/20190914055213_1548540410.run.json
../data_raw/unzipped/2019-09-14/20190914142625_1567713823.run.json
../data_raw/unzipped/2019-09-14/20190914223214_1568475741.run.json
../data_raw/unzipped/2019-09-14/20190914055056_1537154326.run.json
../data_raw/unzipped/2019-09-14/20190914055016_1525508371.run.json
../data_raw/unzipped/2019-09-14/20190914015318_1568438218.run.json
../data_raw/unzipped/2019-09-14/20190914055021_1558159615.run.

../data_raw/unzipped/2019-09-13/20190913131132_1540655602.run.json
../data_raw/unzipped/2019-09-13/20190913060729_1568130919.run.json
../data_raw/unzipped/2019-09-13/20190913062724_1568313457.run.json
../data_raw/unzipped/2019-09-13/20190913001633_1568326564.run.json
../data_raw/unzipped/2019-09-13/20190913032232_1559478387.run.json
../data_raw/unzipped/2019-09-13/20190913132256_1546832074.run.json
../data_raw/unzipped/2019-09-13/20190913132459_1557157633.run.json
../data_raw/unzipped/2019-09-13/20190913033329_1553707891.run.json
../data_raw/unzipped/2019-09-13/20190913131748_1543788923.run.json
../data_raw/unzipped/2019-09-13/20190913014907_1568018647.run.json
../data_raw/unzipped/2019-09-13/20190913014840_1563820094.run.json
../data_raw/unzipped/2019-09-13/20190913210212_1567231210.run.json
../data_raw/unzipped/2019-09-13/20190913032258_1561754449.run.json
../data_raw/unzipped/2019-09-13/20190913210214_1564772788.run.json
../data_raw/unzipped/2019-09-13/20190913032048_1553220571.run.

../data_raw/unzipped/2019-09-13/20190913131751_1543801448.run.json
../data_raw/unzipped/2019-09-13/20190913131513_1522305306.run.json
../data_raw/unzipped/2019-09-13/20190913234515_1565408426.run.json
../data_raw/unzipped/2019-09-13/20190913014951_1556773716.run.json
../data_raw/unzipped/2019-09-13/20190913033803_1568111389.run.json
../data_raw/unzipped/2019-09-13/20190913015917_1566868837.run.json
../data_raw/unzipped/2019-09-13/20190913131632_1523301157.run.json
../data_raw/unzipped/2019-09-13/20190913015621_1566593923.run.json
../data_raw/unzipped/2019-09-13/20190913033049_1560786084.run.json
../data_raw/unzipped/2019-09-13/20190913015941_1562431695.run.json
../data_raw/unzipped/2019-09-13/20190913131206_1543343547.run.json
../data_raw/unzipped/2019-09-13/20190913014853_1565206970.run.json
../data_raw/unzipped/2019-09-13/20190913131204_1542996939.run.json
../data_raw/unzipped/2019-09-13/20190913015625_1565478374.run.json
../data_raw/unzipped/2019-09-13/20190913014958_1558599753.run.

../data_raw/unzipped/2019-09-13/20190913131327_1546121504.run.json
../data_raw/unzipped/2019-09-13/20190913130509_1541389441.run.json
../data_raw/unzipped/2019-09-13/20190913210212_1567974294.run.json
../data_raw/unzipped/2019-09-13/20190913014958_1558595427.run.json
../data_raw/unzipped/2019-09-13/20190913015946_1562049165.run.json
../data_raw/unzipped/2019-09-13/20190913015047_1553275105.run.json
../data_raw/unzipped/2019-09-13/20190913131125_1522996708.run.json
../data_raw/unzipped/2019-09-13/20190913033441_1556089747.run.json
../data_raw/unzipped/2019-09-13/20190913131313_1546133527.run.json
../data_raw/unzipped/2019-09-13/20190913032717_1553785055.run.json
../data_raw/unzipped/2019-09-13/20190913060702_1566222834.run.json
../data_raw/unzipped/2019-09-13/20190913032354_1563810943.run.json
../data_raw/unzipped/2019-09-13/20190913033409_1555830042.run.json
../data_raw/unzipped/2019-09-13/20190913114131_1568229497.run.json
../data_raw/unzipped/2019-09-13/20190913032504_1565494981.run.

../data_raw/unzipped/2019-09-13/20190913040058_1568351592.run.json
../data_raw/unzipped/2019-09-13/20190913015429_1562249915.run.json
../data_raw/unzipped/2019-09-13/20190913033154_1565480753.run.json
../data_raw/unzipped/2019-09-13/20190913032932_1556860511.run.json
../data_raw/unzipped/2019-09-13/20190913015719_1568018036.run.json
../data_raw/unzipped/2019-09-13/20190913131122_1522110151.run.json
../data_raw/unzipped/2019-09-13/20190913033647_1563043624.run.json
../data_raw/unzipped/2019-09-13/20190913015808_1556612245.run.json
../data_raw/unzipped/2019-09-13/20190913131640_1529512512.run.json
../data_raw/unzipped/2019-09-13/20190913033436_1556086868.run.json
../data_raw/unzipped/2019-09-13/20190913015432_1561996471.run.json
../data_raw/unzipped/2019-09-13/20190913032943_1557935215.run.json
../data_raw/unzipped/2019-09-13/20190913033719_1564850658.run.json
../data_raw/unzipped/2019-09-13/20190913131208_1543705198.run.json
../data_raw/unzipped/2019-09-13/20190913132232_1546556771.run.

...

../data_raw/unzipped/2019-02-09/20190209004041_1543805064.run.json
../data_raw/unzipped/2019-02-09/20190209174426_1522529280.run.json
../data_raw/unzipped/2019-02-09/20190209122804_1546604013.run.json
../data_raw/unzipped/2019-02-09/20190209122655_1549114014.run.json
../data_raw/unzipped/2019-02-09/20190209113310_1549729587.run.json
../data_raw/unzipped/2019-02-09/20190209215410_1549509219.run.json
../data_raw/unzipped/2019-02-09/20190209122730_1547074408.run.json
../data_raw/unzipped/2019-02-09/20190209194915_1549689039.run.json
../data_raw/unzipped/2019-02-09/20190209145220_1537748075.run.json
../data_raw/unzipped/2019-02-09/20190209212444_1526884297.run.json
../data_raw/unzipped/2019-02-09/20190209004018_1546128275.run.json
../data_raw/unzipped/2019-02-09/20190209003938_1545450425.run.json
../data_raw/unzipped/2019-02-09/20190209122810_1546881996.run.json
../data_raw/unzipped/2019-02-09/20190209145311_1544827947.run.json
../data_raw/unzipped/2019-02-09/20190209195837_1540798889.run.

../data_raw/unzipped/2019-02-09/20190209212700_1543066506.run.json
../data_raw/unzipped/2019-02-09/20190209194917_1548009505.run.json
../data_raw/unzipped/2019-02-09/20190209003942_1543944672.run.json
../data_raw/unzipped/2019-02-09/20190209212201_1539603456.run.json
../data_raw/unzipped/2019-02-09/20190209042915_1547615406.run.json
../data_raw/unzipped/2019-02-09/20190209195832_1540444676.run.json
../data_raw/unzipped/2019-02-09/20190209004019_1546138619.run.json
../data_raw/unzipped/2019-02-09/20190209145319_1539036667.run.json
../data_raw/unzipped/2019-02-09/20190209003943_1544135672.run.json
../data_raw/unzipped/2019-02-09/20190209004011_1545534092.run.json
../data_raw/unzipped/2019-02-09/20190209195755_1543998521.run.json
../data_raw/unzipped/2019-02-09/20190209215411_1549518286.run.json
../data_raw/unzipped/2019-02-09/20190209174316_1528001726.run.json
../data_raw/unzipped/2019-02-09/20190209205712_1548121789.run.json
../data_raw/unzipped/2019-02-09/20190209174343_1522211766.run.

../data_raw/unzipped/2019-02-09/20190209043029_1548309336.run.json
../data_raw/unzipped/2019-02-09/20190209122640_1545667753.run.json
../data_raw/unzipped/2019-02-09/20190209174318_1546981356.run.json
../data_raw/unzipped/2019-02-09/20190209003936_1545084836.run.json
../data_raw/unzipped/2019-02-09/20190209195801_1545020565.run.json
../data_raw/unzipped/2019-02-09/20190209110713_1549337047.run.json
../data_raw/unzipped/2019-02-09/20190209174422_1547675503.run.json
../data_raw/unzipped/2019-02-09/20190209212451_1527556945.run.json
../data_raw/unzipped/2019-02-09/20190209212402_1545643019.run.json
../data_raw/unzipped/2019-02-09/20190209145438_1547606393.run.json
../data_raw/unzipped/2019-02-09/20190209195804_1541475926.run.json
../data_raw/unzipped/2019-02-09/20190209212145_1523700951.run.json
../data_raw/unzipped/2019-02-09/20190209174314_1549693362.run.json
../data_raw/unzipped/2019-02-09/20190209145228_1539828929.run.json
../data_raw/unzipped/2019-02-09/20190209215415_1549766685.run.

../data_raw/unzipped/2019-02-09/20190209212507_1530634250.run.json
../data_raw/unzipped/2019-02-09/20190209145516_1541682693.run.json
../data_raw/unzipped/2019-02-09/20190209145232_1541387146.run.json
../data_raw/unzipped/2019-02-09/20190209004118_1544980938.run.json
../data_raw/unzipped/2019-02-09/20190209140947_1549518331.run.json
../data_raw/unzipped/2019-02-09/20190209195648_1548560461.run.json
../data_raw/unzipped/2019-02-09/20190209212549_1532871927.run.json
../data_raw/unzipped/2019-02-09/20190209212343_1545576194.run.json
../data_raw/unzipped/2019-02-09/20190209140955_1549502084.run.json
../data_raw/unzipped/2019-02-09/20190209205659_1548030389.run.json
../data_raw/unzipped/2019-02-09/20190209163427_1549600200.run.json
../data_raw/unzipped/2019-02-09/20190209212402_1545645343.run.json
../data_raw/unzipped/2019-02-09/20190209195748_1543052625.run.json
../data_raw/unzipped/2019-02-09/20190209145233_1541858864.run.json
../data_raw/unzipped/2019-02-09/20190209174507_1523148515.run.

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [110]:
output_list

[{'gold_per_floor': [209,
   209,
   209,
   56,
   76,
   107,
   107,
   121,
   148,
   168,
   168,
   202,
   214,
   231,
   231,
   327,
   327,
   347,
   367,
   367,
   384,
   404,
   404],
  'floor_reached': 23,
  'playtime': 579,
  'items_purged': [],
  'score': 205,
  'play_id': '07eac3f5-22cc-42f1-87bd-9d6b69d1626c',
  'local_time': '20190510212451',
  'is_ascension_mode': False,
  'campfire_choices': [{'data': 'Flex', 'floor': 7, 'key': 'SMITH'},
   {'data': 'Demon Form', 'floor': 15, 'key': 'SMITH'}],
  'neow_cost': 'NONE',
  'seed_source_timestamp': 2091687910000,
  'circlet_count': 0,
  'master_deck': ['Strike_R',
   'Strike_R',
   'Strike_R',
   'Strike_R',
   'Strike_R',
   'Defend_R',
   'Defend_R',
   'Defend_R',
   'Defend_R',
   'Bash',
   'Pommel Strike',
   'Flex+1',
   'Searing Blow',
   'Twin Strike',
   'Demon Form+1',
   'Feed',
   'Uppercut',
   'Demon Form',
   'Heavy Blade',
   'Headbutt+1',
   'RitualDagger',
   'Iron Wave',
   'Heavy Blade+1'],
  're